In [1]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai datasets faiss-gpu sentence_transformers

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

In [3]:
from langchain.globals import set_verbose

set_verbose(True)

#🔗 **Origins of CoT Prompting**

- CoT was introduced by Wei et al. in their 2022 paper, [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://browse.arxiv.org/pdf/2201.11903.pdf).

- It's a response to the need for better reasoning in large language models.

💡 **What is CoT Prompting?**
- CoT stands for Chain of Thought Prompting.

- It's about guiding language models through a step-by-step reasoning process.

- The model shows its reasoning, forming a "chain" of thoughts, rather than just giving an answer.

- Mimics human-like thought processes for complex tasks.


### 🎓 **Standard vs. CoT Prompting Example**

<figure>
  <img src="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75" alt="Image Description" style="width:100%">
  <figcaption>Examples of ⟨input, chain of thought, output⟩ triples for arithmetic, commonsense, and symbolic reasoning benchmarks. Chains of thought are highlighted.</figcaption>
  <a href="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75">Image Source</a>
</figure>

- Standard: Direct answer with no reasoning (e.g., "11").

- CoT: Detailed reasoning steps (e.g., "Roger started with 5, added 6 from 2 cans, so 5 + 6 = 11").

### 🚀 **Usage of CoT Prompting**

1. **Enhanced Reasoning**: Breaks down complex problems into simpler steps.

2. **Combination with Few-Shot Prompting**: Uses examples to guide model responses, great for complex tasks.

3. **Interpretability**: Offers a clear view of the model's thought process.

4. **Applications**: Useful in arithmetic, commonsense reasoning, and symbolic tasks.

### ✨ **Impact of CoT Prompting**

- Improves accuracy and insightfulness in responses.

- Focuses on reasoning and interpretability, especially in complex scenarios.

In [5]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 14.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 17.8 MB/s eta 0:00:00
Note: you may need to r

The code below downloads an dataset which has over [1.8 million Chain of Thought examples](https://huggingface.co/datasets/kaist-ai/CoT-Collection).

In [6]:
from datasets import load_dataset

dataset = load_dataset("kaist-ai/CoT-Collection", split="train", trust_remote_code=True)

/home/vscode/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1837928 examples [00:50, 36176.12 examples/s]


In [7]:
 dataset = dataset.remove_columns(['task', 'type'])

 dataset = dataset.shuffle(seed=42)

 dataset = dataset.select(range(10_000))

 dataset = dataset.to_pandas()

 dataset.head()

,source,target,rationale
0,Question: What about Neptune did NASA propose ...,no,The proposed mission to Neptune is not mention...
1,Choose your reply from the options at the end....,yes,The passage talks about the Tucson-Pima County...
2,You are given an original reference as well as...,1,The system generated reference is grammaticall...
3,"Is the premise ""A person on a beach in a green...",yes,The premise describes a situation where there ...
4,Pick the option in line with common sense to a...,C,"The rationale is: ""C, expected because it was ..."


We'll sample just 10,000 of these Chain of Thought examples and save them as a list of dictionaries.

In [8]:
selected_examples = dataset.to_dict(orient='records')

In [9]:
selected_examples[0]

{'source': 'Question: What about Neptune did NASA propose in 2003 in their "Vision Missions Studies"?\n\nIs However, there have been a couple of discussions to launch Neptune missions sooner. a good answer to this question?\n\nOPTIONS:\n- yes\n- no',
 'target': 'no',
 'rationale': 'The proposed mission to Neptune is not mentioned. The only mention of the planet in this excerpt is a note that there have been proposals for such missions, but they are very distant and probably will never happen.'}

In [11]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 16.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 20.3 MB/s

For this example we'll use an open source embedding model from HuggingFace so we don't accumulate a huge bill for OpenAI

In [12]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

/home/vscode/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AssertionError: Torch not compiled with CUDA enabled

To set up a Chain of Thought prompt you will use a `FewShotPromptTemplate` as well as an example selector.

In this example you'll use `MaxMarginalRelevanceExampleSelector`, but you can use any of the example selectors you learned about before.

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

from langchain_community.vectorstores import FAISS

In [ ]:
prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

In [ ]:
examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

There's a large number of examples to embed and add to the vector store. The below cell will take a few minutes to run. It took me ~3 minutes using the free T4 GPU provided on Google colab.

In [ ]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    selected_examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=7,
)

In [ ]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [ ]:
query = """Here's a short story: A lifter uses a barbell, but moves a jump rope\
in a wider arc. The object likely to travel further is (A) the jump rope (B) the\
barbell.

What is the most sensical answer between "a jump rope" and "a barbell"?
"""

In [ ]:
print(mmr_prompt.format(input=query))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0.0)

llm_chain = mmr_prompt | llm | StrOutputParser()

for chunk in llm_chain.stream({"input":query}):
  print(chunk, end="", flush=True)

In [ ]:
query = """
Given the fact that: Inhaling, or breathing in, increases the size of the chest, \
 which decreases air pressure inside the lungs. Answer the question: If Mona is \
 done with a race and her chest contracts, what happens to the amount of air \
 pressure in her lungs increases or decreases?
"""

In [ ]:
for chunk in llm_chain.stream({"input":query}):
  print(chunk, end="", flush=True)